In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
import datetime as dt
import optuna as opt

In [47]:
sp_df = pd.read_csv('data/extended_data_monthly.csv')
sp_df['Date'] =  pd.to_datetime(sp_df['Date'], format='%Y-%m-%d')
sp_df = sp_df.drop(columns=['Open', 'High', 'Low', 'Volume'])

We'll use a one-month lag for the explanatory features. The time features will help introduce any seasonality into a gradient boosting model.

In [51]:
def create_lag_features(df):
    df.copy()
    df = df.shift(periods=1)
    return df

def create_time_features(df):
    df = df.copy()
    df['quarter'] = df['Date'].dt.quarter
    df['month'] = df['Date'].dt.month
    return df

Replacing features with 1-month lags.

In [52]:
sp_df_lag = create_lag_features(sp_df)

In [53]:
sp_df = pd.concat([sp_df[['Close', 'Date']], sp_df_lag.drop(columns=['Close', 'Date'])], axis=1)

In [54]:
sp_df = create_time_features(sp_df)

In [55]:
sp_df.head()

,Close,Date,CPI,E INFL,GDP growth,retail food services,PPI,Risk Premium,QE,Real interest rate,unemployment,oil,M2NS,quarter,month
0,879.82,2002-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,12
1,855.70,2003-01-01,2.376910,1.951158,0.111308,293947.0,4.8,1.264753,725800.50,1.692254,6.0,28.33,5804.0,1,1
2,841.15,2003-02-01,2.597403,2.079012,3.326408,295248.0,6.8,1.196914,724718.40,1.488202,5.8,31.18,5793.5,1,2
3,848.18,2003-03-01,2.980877,2.106538,1.700788,291167.0,9.2,1.177252,721326.00,1.466379,5.9,32.77,5820.8,1,3
4,916.92,2003-04-01,3.020134,1.903343,2.964778,296325.0,11.4,1.151881,723651.25,1.223591,5.9,30.61,5882.1,2,4


Splitting into train and test. Last 12 observations will be forecasted for comparison with ARIMAX.

In [64]:
train = sp_df.iloc[1:-12,:]
test = sp_df.tail(12)

In [57]:
train.head()

,Close,Date,CPI,E INFL,GDP growth,retail food services,PPI,Risk Premium,QE,Real interest rate,unemployment,oil,M2NS,quarter,month
1,855.70,2003-01-01,2.376910,1.951158,0.111308,293947.0,4.8,1.264753,725800.50,1.692254,6.0,28.33,5804.0,1,1
2,841.15,2003-02-01,2.597403,2.079012,3.326408,295248.0,6.8,1.196914,724718.40,1.488202,5.8,31.18,5793.5,1,2
3,848.18,2003-03-01,2.980877,2.106538,1.700788,291167.0,9.2,1.177252,721326.00,1.466379,5.9,32.77,5820.8,1,3
4,916.92,2003-04-01,3.020134,1.903343,2.964778,296325.0,11.4,1.151881,723651.25,1.223591,5.9,30.61,5882.1,2,4
5,963.59,2003-05-01,2.224694,1.989513,2.253233,295600.0,6.0,1.192092,736926.40,1.309618,6.0,25.00,5938.5,2,5


In [58]:
test.head()

,Close,Date,CPI,E INFL,GDP growth,retail food services,PPI,Risk Premium,QE,Real interest rate,unemployment,oil,M2NS,quarter,month
228,4766.18,2021-12-01,6.809003,2.479840,7.094425,646132.0,44.987,1.180230,8648682.25,0.163998,4.2,81.05,21323.7,4,12
229,4515.55,2022-01-01,7.036403,2.620309,-0.560500,639273.0,40.838,1.171871,8723909.40,0.091689,3.9,74.17,21647.9,1,1
230,4373.94,2022-02-01,7.479873,2.241758,-4.532638,650682.0,41.653,1.206808,8820579.50,0.319495,4.0,86.51,21628.4,1,2
231,4530.41,2022-03-01,7.871064,2.627419,-3.129839,659782.0,42.060,1.217189,8897595.50,0.441593,3.8,97.13,21582.3,1,3
232,4131.93,2022-04-01,8.542456,3.057699,-0.919649,671904.0,45.014,1.162330,8933825.00,0.347090,3.6,117.25,21855.9,2,4


In [65]:
train = train.drop(columns=['Date'])
test = test.drop(columns=['Date'])

X_train = train.drop(columns=['Close'])
X_test = test.drop(columns=['Close'])

y_train = train['Close']
y_test = test['Close']

The train test split for time series. We'll use the same test set of 12 months in each split.

In [67]:
tss = TimeSeriesSplit(n_splits=5, test_size=12)

In [94]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = xgb.XGBRegressor(**param)
    score = np.mean(cross_val_score(model, X_train, y_train, cv=tss, scoring='neg_mean_squared_error'))
    return score

In [95]:
study = opt.create_study(direction='maximize', study_name='regression')
study.optimize(objective, n_trials=500)

print('Best hyperparameters:', study.best_params)
print('Best score:', study.best_value)

[I 2023-04-23 02:32:10,422] A new study created in memory with name: regression
[I 2023-04-23 02:32:12,205] Trial 0 finished with value: -674678.2753812643 and parameters: {'max_depth': 10, 'learning_rate': 0.5776992301432907, 'n_estimators': 623, 'min_child_weight': 8, 'gamma': 0.543342720086501, 'subsample': 0.2552124158635877, 'colsample_bytree': 0.6146795699028847, 'reg_alpha': 0.4574821024665635, 'reg_lambda': 0.03289116041054722, 'random_state': 600}. Best is trial 0 with value: -674678.2753812643.
[I 2023-04-23 02:32:14,065] Trial 1 finished with value: -222306.5876499692 and parameters: {'max_depth': 8, 'learning_rate': 0.37941188715141655, 'n_estimators': 298, 'min_child_weight': 1, 'gamma': 0.4020724092122117, 'subsample': 0.8061278270768639, 'colsample_bytree': 0.8172823649576895, 'reg_alpha': 0.19844339561261337, 'reg_lambda': 0.7444565486997324, 'random_state': 207}. Best is trial 1 with value: -222306.5876499692.
[I 2023-04-23 02:32:16,073] Trial 2 finished with value: -2

Best hyperparameters: {'max_depth': 10, 'learning_rate': 0.638081259117982, 'n_estimators': 433, 'min_child_weight': 1, 'gamma': 0.617683305558078, 'subsample': 0.9458649718156338, 'colsample_bytree': 0.5440847211012665, 'reg_alpha': 0.08939913579442578, 'reg_lambda': 0.5916556461409463, 'random_state': 55}
Best score: -174648.23710072745


We use the best XGBoost parameters to forecast on the test set.

In [96]:
xgb_model = xgb.XGBRegressor(**study.best_params)

In [97]:
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5440847211012665, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.617683305558078, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.638081259117982, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=433, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=55, ...)

In [98]:
preds = xgb_model.predict(X_test)

In [99]:
print("Mean squared error:", mean_squared_error(y_test, preds))
print("MAPE:", mean_absolute_percentage_error(y_test, preds))
print("Mean absolute error:", mean_absolute_error(y_test, preds))

Mean squared error: 216242.40547022142
MAPE: 0.0988816639238967
Mean absolute error: 389.83396484375
